In [12]:
from au.com.gegroup.ts.iot_his_reader import IOTHistoryReader
reader = IOTHistoryReader(sqlContext, "iot_history_hybrid", "history")
reader.get_df().show()

+-------------+--------------+-------------------+----------+-------+---------+-----+
|    pointName|      equipRef|           datetime|  levelRef|siteRef|yearMonth|value|
+-------------+--------------+-------------------+----------+-------+---------+-----+
|Chiller-2_STS|Site Chiller 2|1507046744000000000|Site Plant|   Site|  2017-10|  0.0|
|Chiller-2_ENB|Site Chiller 2|1507050417000000000|Site Plant|   Site|  2017-10|  0.0|
|   CDWP-2_ENB|   Site CDWP 2|1507050476000000000|Site Plant|   Site|  2017-10|  0.0|
|   CDWP-2_STS|   Site CDWP 2|1507050489000000000|Site Plant|   Site|  2017-10|  0.0|
|   CHWP-2_ENB|   Site CHWP 2|1507050535000000000|Site Plant|   Site|  2017-10|  0.0|
|   CHWP-2_STS|   Site CHWP 2|1507050551000000000|Site Plant|   Site|  2017-10|  0.0|
| Boiler-1_STS| Site Boiler 1|1507544565000000000|Site Plant|   Site|  2017-10|  0.0|
|    HWP-8_STS|    Site HWP 8|1507544634000000000|Site Plant|   Site|  2017-10|  0.0|
| Boiler-2_STS| Site Boiler 2|1507545158000000000|Site

In [2]:
from au.com.gegroup.ts.datetime.utils import *
import pandas as pd
swt_ts = reader.metadata("supply and water and temp and sensor and equipRef != 'Site Chiller Plant'"). \
history(last(pd.Timedelta("90 days"))).read()
lwt_ts = reader.metadata("return and water and temp and sensor and equipRef != 'Site Chiller Plant'"). \
history(last(pd.Timedelta("90 days"))).read()

Query = 
select datetime as time, value as value , pointName as pointName, equipRef as equipRef, levelRef as levelRef, siteRef as siteRef from history  where datetime >= 1506427915704737000 and datetime <= 1514203915704737000 and   (pointName = 'Boiler-1_LWT' or pointName = 'Boiler-2_LWT' or pointName = 'CT1_LWT' or pointName = 'CT2_LWT' or pointName = 'Chiller-1_ECWT' or pointName = 'Chiller-1_LCHWT' or pointName = 'Chiller-2_ECWT' or pointName = 'Chiller-2_LCHWT' or pointName = 'Chiller-3_ECWT' or pointName = 'Chiller-3_LCHWT' or pointName = 'Tenant-CT-2_LWT') and equipRef != 'Site Chiller Plant'
Query = 
select datetime as time, value as value , pointName as pointName, equipRef as equipRef, levelRef as levelRef, siteRef as siteRef from history  where datetime >= 1506427916102799000 and datetime <= 1514203916102799000 and   (pointName = 'CT1_EWT' or pointName = 'CT2_EWT' or pointName = 'Chiller-1_ECHWT' or pointName = 'Chiller-1_LCWT' or pointName = 'Chiller-2_ECHWT' or pointName = '

In [3]:
swt_ts.show()

+-------------------+------------------+---------------+----------------+----------+-------+
|               time|             value|      pointName|        equipRef|  levelRef|siteRef|
+-------------------+------------------+---------------+----------------+----------+-------+
|1507679100000000000| 24.50189971923828|   Boiler-1_LWT|   Site Boiler 1|Site Plant|   Site|
|1507679100000000000| 24.92799949645996|   Boiler-2_LWT|   Site Boiler 2|Site Plant|   Site|
|1507679700000000000|20.811500549316406|Tenant-CT-2_LWT|Site Tenant-CT-2|Site Plant|   Site|
|1507679880000000000|15.946599960327148|Chiller-1_LCHWT|  Site Chiller 1|Site Plant|   Site|
|1507679880000000000|  22.9867000579834|Chiller-2_LCHWT|  Site Chiller 2|Site Plant|   Site|
|1507679880000000000| 18.11870002746582|Chiller-3_LCHWT|  Site Chiller 3|Site Plant|   Site|
|1507680000000000000|24.497900009155273|   Boiler-1_LWT|   Site Boiler 1|Site Plant|   Site|
|1507680000000000000|24.908199310302734|   Boiler-2_LWT|   Site Boiler

In [4]:
lwt_ts.show()

+-------------------+------------------+---------------+----------------+----------+-------+
|               time|             value|      pointName|        equipRef|  levelRef|siteRef|
+-------------------+------------------+---------------+----------------+----------+-------+
|1507679100000000000|27.546600341796875|Tenant-CT-1_EWT|Site Tenant-CT-1|Site Plant|   Site|
|1507679100000000000|20.219100952148438|Tenant-CT-2_EWT|Site Tenant-CT-2|Site Plant|   Site|
|1507679880000000000|14.758399963378906|Chiller-1_ECHWT|  Site Chiller 1|Site Plant|   Site|
|1507679880000000000|22.425500869750977|Chiller-2_ECHWT|  Site Chiller 2|Site Plant|   Site|
|1507679880000000000|18.954700469970703|Chiller-3_ECHWT|  Site Chiller 3|Site Plant|   Site|
|1507680000000000000|14.768099784851074|Chiller-1_ECHWT|  Site Chiller 1|Site Plant|   Site|
|1507680000000000000|22.426700592041016|Chiller-2_ECHWT|  Site Chiller 2|Site Plant|   Site|
|1507680000000000000|18.955299377441406|Chiller-3_ECHWT|  Site Chiller

In [5]:
from au.com.gegroup.ts.spark.utils import join_keys
joined_ts = lwt_ts.leftJoin(swt_ts, tolerance='1M', key=join_keys(), left_alias="lwt", right_alias="swt")
joined_ts.show()

+-------------------+------------------+---------------+----------------+----------+-------+------------------+---------------+
|               time|         lwt_value|  lwt_pointName|        equipRef|  levelRef|siteRef|         swt_value|  swt_pointName|
+-------------------+------------------+---------------+----------------+----------+-------+------------------+---------------+
|1507679100000000000|27.546600341796875|Tenant-CT-1_EWT|Site Tenant-CT-1|Site Plant|   Site|              null|           null|
|1507679100000000000|20.219100952148438|Tenant-CT-2_EWT|Site Tenant-CT-2|Site Plant|   Site|              null|           null|
|1507679880000000000|14.758399963378906|Chiller-1_ECHWT|  Site Chiller 1|Site Plant|   Site|15.946599960327148|Chiller-1_LCHWT|
|1507679880000000000|22.425500869750977|Chiller-2_ECHWT|  Site Chiller 2|Site Plant|   Site|  22.9867000579834|Chiller-2_LCHWT|
|1507679880000000000|18.954700469970703|Chiller-3_ECHWT|  Site Chiller 3|Site Plant|   Site| 18.11870002

In [6]:
from au.com.gegroup.ts.spark.utils import *
filtered_df = joined_ts.filter(filter_null("lwt", "swt"))

In [7]:
from pyspark.sql.functions import udf, col
# user's rule to calculate deviation

def delta(firstValue, secondValue):
    return firstValue - secondValue
    
# spark's udf calling user's rule
delta_udf = udf(lambda lwt_val, swt_val: delta(lwt_val, swt_val))

rule_df = filtered_df.withColumn("delta", delta_udf(get_value_col("lwt"), get_value_col("swt")))
rule_df.show()

+-------------------+------------------+---------------+----------------+----------+-------+------------------+---------------+-------------------+
|               time|         lwt_value|  lwt_pointName|        equipRef|  levelRef|siteRef|         swt_value|  swt_pointName|              delta|
+-------------------+------------------+---------------+----------------+----------+-------+------------------+---------------+-------------------+
|1507679880000000000|14.758399963378906|Chiller-1_ECHWT|  Site Chiller 1|Site Plant|   Site|15.946599960327148|Chiller-1_LCHWT|-1.1881999969482422|
|1507679880000000000|22.425500869750977|Chiller-2_ECHWT|  Site Chiller 2|Site Plant|   Site|  22.9867000579834|Chiller-2_LCHWT|-0.5611991882324219|
|1507679880000000000|18.954700469970703|Chiller-3_ECHWT|  Site Chiller 3|Site Plant|   Site| 18.11870002746582|Chiller-3_LCHWT| 0.8360004425048828|
|1507680000000000000|14.768099784851074|Chiller-1_ECHWT|  Site Chiller 1|Site Plant|   Site|15.959600448608398|C

In [8]:
rule_df.createOrReplaceTempView("ruleDf")

In [10]:
sqlContext.sql("select equipRef, min(delta), max(delta) from ruleDf group by equipRef order by equipRef ASC").show()

+----------------+--------------------+-------------------+
|        equipRef|          min(delta)|         max(delta)|
+----------------+--------------------+-------------------+
|  Site Chiller 1|-0.00629997253417...|  7.227100372314453|
|  Site Chiller 2| -0.4890003204345703|-0.8651008605957031|
|  Site Chiller 3|  0.4233999252319336|  9.285099029541016|
|Site Tenant-CT-2|-0.01070022583007...|0.09740066528320312|
+----------------+--------------------+-------------------+



In [11]:
from au.com.gegroup.ts.writer import Writer
# Writer(dataframe, dataset, row_keys)
writer = Writer(rule_df, "hybrid_chiller_delta_oct2017", "time,swt_pointName,lwt_pointName,equipRef")
cols = ["time", "equipRef", "swt_pointName", "lwt_pointName", "swt_value", "lwt_value", "delta"]
writer.cols(cols).mode("overwrite").write()

In [13]:
delta_reader = IOTHistoryReader(sqlContext, "metaless_chiller_delta_oct2017", "deltaDF")
delta_reader.get_df().show()

+------------------+------------------+------------------+-------------------+-------------------+---------------+---------------+
|          equipRef|         swt_value|         lwt_value|              delta|               time|  swt_pointName|  lwt_pointName|
+------------------+------------------+------------------+-------------------+-------------------+---------------+---------------+
|    Site Chiller 1|15.946599960327148|14.758399963378906|-1.1881999969482422|1507679880000000000|Chiller-1_LCHWT|Chiller-1_ECHWT|
|    Site Chiller 2|  22.9867000579834|22.425500869750977|-0.5611991882324219|1507679880000000000|Chiller-2_LCHWT|Chiller-2_ECHWT|
|Site Chiller Plant|22.670900344848633| 25.53059959411621|  2.859699249267578|1507679880000000000| Chiller-3_ECWT|        CT2_EWT|
|Site Chiller Plant|22.670900344848633| 25.53059959411621|  2.859699249267578|1507679880000000000| Chiller-3_ECWT| Chiller-1_LCWT|
|Site Chiller Plant|22.670900344848633| 25.53059959411621|  2.859699249267578|15076